# Softmax

## loss

In [ ]:
from torch import nn, tensor, max
import numpy as np

In [ ]:
# Cross entropy example (np only)
# One hot
# 0: 1 0 0
# 1: 0 1 0
# 2: 0 0 1
Y = np.array([1, 0, 0])
Y_pred1 = np.array([0.7, 0.2, 0.1])  # small loss
Y_pred2 = np.array([0.1, 0.3, 0.6])  # large loss

# calculate loss manually
print(f'Loss1: {np.sum(-Y * np.log(Y_pred1)):.4f}')
print(f'Loss2: {np.sum(-Y * np.log(Y_pred2)):.4f}')

Loss1: 0.3567
Loss2: 2.3026


In [ ]:
# Softmax + CrossEntropy (logSoftmax + NLLLoss)
loss = nn.CrossEntropyLoss()

In [ ]:
# target is of size nBatch
# each element in target has to have 0 <= value < nClasses (0-2)
# Input is class, not one-hot!!!
Y = tensor([0], requires_grad=False)  # means class 0 is correct

# input is of size nBatch x nClasses = 1 x 4
# Y_pred are logits (not softmax)!!!
Y_pred1 = tensor([[2.0, 1.0, 0.1]])  # should be small, since [0] is larger than the rest
Y_pred2 = tensor([[0.5, 2.0, 0.3]])

l1 = loss(Y_pred1, Y)
l2 = loss(Y_pred2, Y)

print(f'PyTorch Loss1: {l1.item():.4f} \nPyTorch Loss2: {l2.item():.4f}')
print(f'Y_pred1: {max(Y_pred1.data, 1)[1].item()}')
print(f'Y_pred2: {max(Y_pred2.data, 1)[1].item()}')

PyTorch Loss1: 0.4170 
PyTorch Loss2: 1.8406
Y_pred1: 0
Y_pred2: 1


In [ ]:
# target is of size nBatch
# each element in target has to have 0 <= value < nClasses (0-2)
# Input is class, not one-hot
Y = tensor([2, 0, 1], requires_grad=False)

# input is of size nBatch x nClasses = 2 x 4
# Y_pred are logits (not softmax)
Y_pred1 = tensor([[0.1, 0.2, 0.9],
                  [1.1, 0.1, 0.2],
                  [0.2, 2.1, 0.1]])  # should be small

Y_pred2 = tensor([[0.8, 0.2, 0.3],
                  [0.2, 0.3, 0.5],
                  [0.2, 0.2, 0.5]])  # should be large

l1 = loss(Y_pred1, Y)
l2 = loss(Y_pred2, Y)
print(f'Batch Loss1:  {l1.item():.4f} \nBatch Loss2: {l2.data:.4f}')

Batch Loss1:  0.4966 
Batch Loss2: 1.2389


## MNIST

http://yann.lecun.com/exdb/mnist/

http://cs231n.github.io/neural-networks-3/#sgd

https://blog.csdn.net/u010089444/article/details/76725843

https://pytorch.org/docs/stable/torchvision/transforms.html?highlight=transforms

https://pytorch.org/docs/stable/nn.functional.html?highlight=functional%20relu#torch.nn.functional.relu

https://pytorch.org/docs/stable/nn.html#torch.nn.ReLU


In [ ]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms  # transforms: common image transformations
import torch.nn.functional as F
import time

In [ ]:
# Training settings
batch_size = 64
device = 'cuda' if cuda.is_available() else 'cpu'
# device = 'cpu'
print(f'Training MNIST Model on {device}\n{"=" * 44}')

Training MNIST Model on cuda


In [ ]:
# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

In [ ]:
# Data Loader (Input Pipeline)
train_loader = data.DataLoader(dataset=train_dataset,
                               batch_size=batch_size,  # 64
                               shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset,
                              batch_size=batch_size,
                              shuffle=False)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l1(x))  # activation function
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)

In [ ]:
model = Net()
model.to(device)
criterion = nn.CrossEntropyLoss()
# SGD with momentum is method which helps accelerate gradients vectors in the right directions
# thus leading to faster converging
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [ ]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)  # Cross Entropy Loss
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')


In [ ]:
since = time.time()
for epoch in range(1, 10):
    epoch_start = time.time()
    train(epoch)
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Training time: {m:.0f}m {s:.0f}s')
    test()
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Testing time: {m:.0f}m {s:.0f}s')

m, s = divmod(time.time() - since, 60)
print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')

Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 2.295890
Train Epoch: 1 | Batch Status: 12800/60000 (21%) | Loss: 2.307435
Train Epoch: 1 | Batch Status: 25600/60000 (43%) | Loss: 2.282576
Train Epoch: 1 | Batch Status: 38400/60000 (64%) | Loss: 2.260816
Train Epoch: 1 | Batch Status: 51200/60000 (85%) | Loss: 2.049404
Training time: 0m 7s
Test set: Average loss: 0.0227, Accuracy: 5633/10000 (56%)
Testing time: 0m 8s
Train Epoch: 2 | Batch Status: 0/60000 (0%) | Loss: 1.280056
Train Epoch: 2 | Batch Status: 12800/60000 (21%) | Loss: 0.837074
Train Epoch: 2 | Batch Status: 25600/60000 (43%) | Loss: 0.550821
Train Epoch: 2 | Batch Status: 38400/60000 (64%) | Loss: 0.487448
Train Epoch: 2 | Batch Status: 51200/60000 (85%) | Loss: 0.378238
Training time: 0m 7s
Test set: Average loss: 0.0068, Accuracy: 8741/10000 (87%)
Testing time: 0m 8s
Train Epoch: 3 | Batch Status: 0/60000 (0%) | Loss: 0.452543
Train Epoch: 3 | Batch Status: 12800/60000 (21%) | Loss: 0.423461
Train Epoch: 3 | Batch 